# SystemVerilog coverage using Athena AWS

# To run this demo:

- Create yourself an AWS account - easy and free
- Add an IAM user with S3 full permissions and Athena full permissions
- Fill in the IAM user credentials below

In [ ]:
%set_env AWS_ACCESS_KEY_ID add_your_key_id_here
%set_env AWS_SECRET_ACCESS_KEY add_your_secret_access_key_here
%set_env AWS_DEFAULT_REGION us-east-1

# let's test that your credentials are working correct
! aws s3api list-buckets

# The raw data:
- transactions_log - is the log of the transactions on an interface
- type_information - is the type of each field printed in the transaction_log

This information is generated by the following example: https://www.edaplayground.com/x/2hJC

In [ ]:
transaction_log = \
"""# Time:    0, dir: RD, addr: 1490070710, burst: WRAP, len: 15, id:  3, lock: EXCLUSIVE,
# Time:   26, dir: WR, addr:  165377426, burst: INCR, len:  0, id: 12, lock: NORMAL,
# Time:   61, dir: WR, addr: 2328599037, burst: INCR, len:  8, id: 13, lock: NORMAL,
# Time:  110, dir: RD, addr: 2328599037, burst: WRAP, len:  7, id:  5, lock: EXCLUSIVE,
# Time:  133, dir: RD, addr:  165377426, burst: WRAP, len:  3, id:  7, lock: NORMAL,
# Time:  181, dir: WR, addr: 1490070710, burst: WRAP, len:  6, id: 11, lock: EXCLUSIVE,
# Time:  207, dir: RD, addr: 1490070710, burst: FIXED, len: 15, id:  5, lock: EXCLUSIVE,
# Time:  231, dir: RD, addr: 1490070710, burst: INCR, len:  6, id:  2, lock: NORMAL,
# Time:  252, dir: RD, addr:  165377426, burst: WRAP, len:  9, id:  0, lock: NORMAL,
# Time:  290, dir: WR, addr: 2328599037, burst: WRAP, len: 10, id: 11, lock: NORMAL,
# Time:  300, dir: WR, addr: 1490070710, burst: INCR, len:  7, id:  4, lock: EXCLUSIVE,
# Time:  313, dir: WR, addr: 3668650794, burst: WRAP, len: 12, id:  8, lock: NORMAL,
# Time:  358, dir: RD, addr: 3668650794, burst: INCR, len: 12, id:  7, lock: NORMAL,
# Time:  403, dir: WR, addr: 3668650794, burst: WRAP, len: 10, id: 11, lock: NORMAL,
# Time:  445, dir: RD, addr: 2328599037, burst: INCR, len:  9, id:  2, lock: EXCLUSIVE,
"""

type_information = \
"""# Transaction meta: dir: enum{RD=32'sd0,WR=32'sd1}axi_vip::dir_t, addr:          32, burst: enum{FIXED=32'sd0,INCR=32'sd1,WRAP=32'sd2}axi_vip::burst_t, len:           4, id:           4, lock: enum{NORMAL=32'sd0,EXCLUSIVE=32'sd1,LOCKED=32'sd2}axi_vip::lock_t"""

# Prepare data and upload 

## Package imports

In [ ]:
# import some general purpose packages
import os
import sys
import re
import json
import subprocess
import time
import pandas

# python package for AWS access
import boto3

## Create files to upload

- transactions.log - contains the transactions as shown above
- types.json - contains the values for each enumeration used
- columns.csv - contains information about the type of each column

All these files will be uploaded to S3, and then turned into Athena tables. Note that they all have different formats that Athena can digest.

In [ ]:
script_dir = os.getcwd()

transactions_filename = script_dir + "/simple_tb/test1/axi_master_1/log/transactions.log"
columns_filename = script_dir + "/simple_tb/test1/axi_master_1/columns/columns.csv"
types_filename = script_dir + "/simple_tb/types_info/types.json"

for path in [transactions_filename, columns_filename, types_filename]:
    os.makedirs(os.path.dirname(path)) 

transactions_file = open(transactions_filename, "w")
columns_file = open(columns_filename, "w")
types_file = open(types_filename, "w")

## Extract type/column information

Turn the "type_information" variable into:
- A file that has all enum values nicely organized in rows, 
- Another file that maps column name (i.e. "dir") to type (i.e. "dir_t")
- A variable that stores SQL column definitions (i.e. dir -> string, len -> smallint)

Also write the transactions into a file without any processing.

In [ ]:
type_information = type_information.split("# Transaction meta: ",1)[1]
columns_defs = ""

for type in type_information.split(", "):
    if re.match('.*enum', type):
        match = re.match('([^:]*): enum\{(.*)\}([^,]*)', type)
        if match:
            column_name = match.group(1)
            enum_type_name = match.group(3)
            columns_defs = columns_defs + "`" + column_name + "` string,\n" 
            columns_file.write(column_name + "," + enum_type_name + ",0\n")
            for enum_key_value in match.group(2).split(','):
                match = re.match('([^=]+)=.*([0-9]+)', enum_key_value)
                if match:
                    entry = {}
                    entry['enum_type_name'] = enum_type_name
                    entry['enum_string'] = match.group(1)
                    entry['enum_int'] = match.group(2)
                    types_file.write(json.dumps(entry) + "\n")
    else:
        match = re.match('([^ ]+): *([0-9]+)$', type)
        if match:
            column_name = match.group(1)
            width = match.group(2)
            width_int = int(width)
            columns_file.write(column_name + ",\t\tint,\t\t" + width + "\n")
            if width_int <= 15:
                columns_defs = columns_defs + "`" + column_name + "` smallint,\n"
            elif width_int <= 31:
                columns_defs = columns_defs + "`" + column_name + "` int,\n"
            else:
                columns_defs = columns_defs + "`" + column_name + "` bigint,\n"

        else:
            print("error parsing meta data: int type: " + type + " doesn't have a width field")

transactions_file.write(transaction_log)

transactions_file.close()
columns_file.close()
types_file.close()

columns_defs = columns_defs[:-2]

## Upload files to S3

In [ ]:
%%bash

aws s3 mb s3://coverage-demo/
aws s3 sync simple_tb/ s3://coverage-demo/simple_tb --delete

# Create database structure

## Create a blocking query function

A thin wrapper around boto3 to run a query and wait for it to finish

In [ ]:
def run_query(query, database, s3_output):
    client = boto3.client('athena', 
                          aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                          aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])
    
    if database:
        response = client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={
                'Database': database
            },
            ResultConfiguration={
                'OutputLocation': s3_output,
            }
        )
    else:
        response = client.start_query_execution(
            QueryString=query,
            ResultConfiguration={
                'OutputLocation': s3_output,
            }
        )
    print('Execution ID: ' + response['QueryExecutionId'])

    # wait for query to finish
    while True:
        execution = client.get_query_execution(QueryExecutionId=response['QueryExecutionId'])
        if execution['QueryExecution']['Status']['State'] in ['SUCCEEDED', 'FAILED']:
            print(execution['QueryExecution']['Status']['State'])
            break
        else:
            print(execution['QueryExecution']['Status']['State'])
            time.sleep(2) 

    response = client.get_query_results(QueryExecutionId=response['QueryExecutionId'])
    if response['ResultSet']['Rows']:
        row_num = 0;
        for raw_row in response['ResultSet']['Rows']:
            row = list()
            for cell in raw_row['Data']:
                row.append(list(cell.values())[0])
            if row_num is 0:
                df = pandas.DataFrame(columns=row)
            else:
                df.loc[row_num] = row
            row_num += 1
        return df

## Create tables

first clean the database, then recreate it all the tables in it. Note that they're created from the files uploaded to S3.

In [ ]:
s3_base = 's3://coverage-demo/'
s3_output = 's3://coverage-demo/results/'
database = 'coverage_demo'
        
drop_db = "DROP DATABASE IF EXISTS %s CASCADE;" % (database)

create_db = "CREATE DATABASE %s;" % (database)

create_enum_tbl = \
    """CREATE EXTERNAL TABLE %s.%s (
    `enum_type_name` string,
    `enum_string` string,
    `enum_int` int
     )
     ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
     WITH SERDEPROPERTIES (
         'serialization.format' = '1'
     ) LOCATION '%s'
     TBLPROPERTIES ('has_encrypted_data'='false');""" % ( database, "enums_info", s3_base + "/simple_tb/types_info/" )

print(create_enum_tbl + "\n")

create_columns_tbl = \
    """CREATE EXTERNAL TABLE %s.%s (
    `column_name` string,
    `column_type` string,
    `column_width` int
     )
     ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
     WITH SERDEPROPERTIES (
         'separatorChar' = ','
     ) LOCATION '%s'
     TBLPROPERTIES ('has_encrypted_data'='false');""" % ( database, "columns_info", s3_base + "simple_tb/test1/axi_master_1/columns/" )

print(create_columns_tbl + "\n")

create_tr_tbl = \
    """CREATE EXTERNAL TABLE %s.%s (
    `time` bigint,
    %s
     )
     ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
     WITH SERDEPROPERTIES (  
         'input.regex'='# Time: *([^ ^,]*), dir: *([^ ^,]*), addr: *([^ ^,]*), burst: *([^ ^,]*), len: *([^ ^,]*), id: *([^ ^,]*), lock: *([^ ^,]*),' 
     ) LOCATION '%s'
     TBLPROPERTIES ('has_encrypted_data'='false');""" % ( database, "axi_if1_transactions", columns_defs, s3_base + "simple_tb/test1/axi_master_1/log/" )

print(create_tr_tbl + "\n")

for query in [drop_db, create_db]:
    run_query(query, "", s3_output)

for query in [create_enum_tbl, create_columns_tbl, create_tr_tbl]:
    run_query(query, database, s3_output)

# Run queries

## Interrupted RMW query

Looking for exclusive read-exclusive write pair with a write in between

In [ ]:
interrupted_rmw = """select first_tr.addr as addr, first_tr.time as read_time,  min(middle_tr.time) as interrupted_at, min(second_tr.time) as write_time
from (
    select row_number() over () as num, inner1.time, inner1.addr, inner1.dir, inner1.lock from
        axi_if1_transactions inner1
    where
        inner1.dir = 'WR' or
        inner1.lock = 'EXCLUSIVE'
    order by inner1.addr, inner1.time
    ) first_tr,
    (
    select row_number() over () as num, inner1.time, inner1.addr, inner1.dir, inner1.lock from
        axi_if1_transactions inner1
    where
        inner1.dir = 'WR' or
        inner1.lock = 'EXCLUSIVE'
    order by inner1.addr, inner1.time
    ) second_tr,
    (
    select row_number() over () as num, inner1.time, inner1.addr, inner1.dir, inner1.lock from
        axi_if1_transactions inner1
    where
        inner1.dir = 'WR' or
        inner1.lock = 'EXCLUSIVE'
    order by inner1.addr, inner1.time
    ) middle_tr
where first_tr.addr = second_tr.addr and
         second_tr.addr = first_tr.addr and
     first_tr.lock = 'EXCLUSIVE' and
     second_tr.lock = 'EXCLUSIVE' and
     first_tr.dir = 'RD' and
     second_tr.dir = 'WR' and
     middle_tr.dir = 'WR' and
     first_tr.num < middle_tr.num and
     middle_tr.num < second_tr.num
group by 1,2;"""

for query in [interrupted_rmw]:
    df = run_query(query, database, s3_output)

df

## Burst cross RD/WR

In [ ]:
burst_x_dir = """select distinct expected_values.burst, expected_values.dir, if(axi_if1_transactions.burst is not null, 'TRUE', 'FALSE') as covered from axi_if1_transactions right outer join
((
  select enums_info.enum_string as burst from enums_info,
  (
    select columns_info.column_type from columns_info 
    where column_name = 'burst'
    ) column_meta 
  where column_meta.column_type = enums_info.enum_type_name
  ) enum1_values
cross join (
  select enums_info.enum_string as dir from enums_info,
  (
    select columns_info.column_type from columns_info 
    where column_name = 'dir'
    ) column_meta 
  where column_meta.column_type = enums_info.enum_type_name
  ) enum2_values
) expected_values 
on expected_values.burst = axi_if1_transactions.burst and
   expected_values.dir = axi_if1_transactions.dir
order by expected_values.burst, expected_values.dir"""
    
for query in [burst_x_dir]:
    df = run_query(query, database, s3_output)

df